In [2]:
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Вичитуємо [Smoke Detection](https://www.kaggle.com/datasets/deepcontractor/smoke-detection-dataset/) датасет

[Real-time Smoke Detection with AI-based Sensor Fusion](https://www.hackster.io/stefanblattmann/real-time-smoke-detection-with-ai-based-sensor-fusion-1086e6)

1) Load data, display column names and dataset size

In [4]:
smoke_train = pd.read_csv("../lab1/smoke_detection_iot.csv")
smoke_train.shape

(62630, 16)

In [5]:
# показуємо назви колонок
print(smoke_train.keys())

Index(['Index', 'UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]',
       'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5',
       'NC0.5', 'NC1.0', 'NC2.5', 'CNT', 'Fire Alarm'],
      dtype='object')


In [12]:
# змінюємо тип даних на спільний для всіх колонок
smoke_train['TVOC[ppb]'] = smoke_train['TVOC[ppb]'].astype(float)
smoke_train['eCO2[ppm]'] = smoke_train['eCO2[ppm]'].astype(float)
smoke_train['Raw H2'] = smoke_train['Raw H2'].astype(float)
smoke_train['Raw Ethanol'] = smoke_train['Raw Ethanol'].astype(float)
smoke_train['CNT'] = smoke_train['CNT'].astype(float)
smoke_train['Fire Alarm'] = smoke_train['Fire Alarm'].astype(float)

smoke_train = smoke_train.drop(columns=['Index', 'UTC', 'CNT'])
smoke_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature[C]  62630 non-null  float64
 1   Humidity[%]     62630 non-null  float64
 2   TVOC[ppb]       62630 non-null  float64
 3   eCO2[ppm]       62630 non-null  float64
 4   Raw H2          62630 non-null  float64
 5   Raw Ethanol     62630 non-null  float64
 6   Pressure[hPa]   62630 non-null  float64
 7   PM1.0           62630 non-null  float64
 8   PM2.5           62630 non-null  float64
 9   NC0.5           62630 non-null  float64
 10  NC1.0           62630 non-null  float64
 11  NC2.5           62630 non-null  float64
 12  Fire Alarm      62630 non-null  float64
dtypes: float64(13)
memory usage: 6.2 MB


### Повнозв'язна нейронна мережа MLP (Multilayer Perception)

In [13]:
num_classes = 10

X = smoke_train.drop(['Fire Alarm'], axis = 1)
y = smoke_train['Fire Alarm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

50104 train samples
12526 test samples


In [14]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [16]:
scaled_X_train = scaler.transform(X_train) 
scaled_X_test = scaler.transform(X_test) 

In [17]:
model = keras.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=(X_train.shape[-1], )),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation="sigmoid"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation="sigmoid"),
    ])
model.summary()

/home/ikripaka/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)               │      3,328 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_1           │ (None, 256)               │      1,024 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_4 (Dense)                 │ (None, 128)               │     32,896 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_1 (Dropout)             │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ (None, 1)                 │        129 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 37,377 (146.00 KB)

 Trainable params: 36,865 (144.00 KB)

 Non-trainable params: 512 (2.00 KB)

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)
model_1_fit = model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    verbose=2,
    epochs=30, 
    batch_size=128
)

Epoch 1/30
392/392 - 3s - 7ms/step - accuracy: 0.8204 - loss: 0.4017 - val_accuracy: 0.8496 - val_loss: 0.3724
Epoch 2/30
392/392 - 1s - 2ms/step - accuracy: 0.8924 - loss: 0.2447 - val_accuracy: 0.8744 - val_loss: 0.2535
Epoch 3/30
392/392 - 1s - 4ms/step - accuracy: 0.9081 - loss: 0.2125 - val_accuracy: 0.9197 - val_loss: 0.1942
Epoch 4/30
392/392 - 1s - 3ms/step - accuracy: 0.9170 - loss: 0.1950 - val_accuracy: 0.9226 - val_loss: 0.1809
Epoch 5/30
392/392 - 1s - 2ms/step - accuracy: 0.9237 - loss: 0.1828 - val_accuracy: 0.9330 - val_loss: 0.1707
Epoch 6/30
392/392 - 1s - 3ms/step - accuracy: 0.9298 - loss: 0.1719 - val_accuracy: 0.9166 - val_loss: 0.1903
Epoch 7/30
392/392 - 1s - 3ms/step - accuracy: 0.9336 - loss: 0.1655 - val_accuracy: 0.9280 - val_loss: 0.1626
Epoch 8/30
392/392 - 1s - 3ms/step - accuracy: 0.9390 - loss: 0.1554 - val_accuracy: 0.9433 - val_loss: 0.1429
Epoch 9/30
392/392 - 1s - 3ms/step - accuracy: 0.9431 - loss: 0.1466 - val_accuracy: 0.9303 - val_loss: 0.1741
E